In [24]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [32]:
filepath_train = os.path.join(os.path.dirname(os.getcwd()),'dataset','train.csv')
df = pd.read_csv(filepath_train)
X_test_full = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'dataset','test.csv'))

X_test_full.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)

# 直接去掉缺失值的行
df.dropna(axis=0, inplace=True)

y = df['emission']
train_set = df.drop(['ID_LAT_LON_YEAR_WEEK','emission'],axis=1)
train_set
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train_set, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [34]:
# 处理种类列
# 对于树型模型可以直接ordinal encoding 顺序编码
from sklearn.preprocessing import OrdinalEncoder

# Select categorical columns with relatively low cardinalitynumber of unique values
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]


numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

ordinal_encoder = OrdinalEncoder()

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

encoder = OrdinalEncoder()
encoder.fit_transform(X_train[categorical_cols])


array([], shape=(350, 0), dtype=float64)

In [35]:
from sklearn.metrics import mean_squared_error
import math 

def cal_RMSE(y_pred, y_true):
    mse = mean_squared_error(y_pred, y_true)
    rmse = math.sqrt(mse)
    return rmse

In [38]:
# 使用决策树,随机森林,XGBoost三个模型
from sklearn.tree import DecisionTreeRegressor # 决策回归树
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


clf  = DecisionTreeRegressor() 
clf2 = RandomForestRegressor(n_estimators=100, random_state=0)
clf3 = XGBRegressor(n_estimators=500)

clf = clf.fit(X_train,y_train) 
clf2 = clf2.fit(X_train,y_train) 
clf3 = clf3.fit(X_train,y_train) 

y_pred1 = clf.predict(X_valid_full)
y_pred2 = clf2.predict(X_valid_full)
y_pred3 = clf3.predict(X_valid_full)


model_rmse = cal_RMSE(y_pred1,y_valid )
model_rmse2 = cal_RMSE(y_pred2,y_valid )
model_rmse3 = cal_RMSE(y_pred3,y_valid )
print(f'model1(DecisionTree), rmse ={model_rmse}')
print(f'model2(Random Forest), rmse ={model_rmse2}')
print(f'model3(XGBoost), rmse ={model_rmse3}')



rmse =102.08969201274135
